In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.formula.api as smf
from sklearn import linear_model

## 1. Read the data
data could be found on: https://github.com/picniclin/NYC_yl5240/tree/master/data
Data source is from Geolytics: Neighborhood Change Database (NACD)
exported dataset geolytics_nyc_census_1990_2010.csv has been uploaded on: https://github.com/picniclin/NYC_yl5240/blob/master/data/geolytics_nyc_census_1990_2010.csv


- df1: 2077 rows, with all varialbes for analysis, having dropped rows with 0 values of income, rent, and entropy index in the 2168 tracts. The clean version of geolytics_nyc_census_1990_2010.csv
- df_normed0: 2077 rows, normalized on df1
- df2: 1770 rows, drop outliers, based on df1
- df_normed: 1770 rows, normalized on df2


### df1: civic_census.csv


In [2]:
df1 = pd.read_csv('civic_census.csv')

In [3]:
df1.drop(['Unnamed: 0'],axis = 1, inplace = True)
df1.describe()

,tract,entropy_index_9,entropy_index_0,entropy_index_1,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,...,rent_burden_9_0,entropy_0_1,rent_0_1,inc_0_1,rent_units_0_1,rent_burden_0_1,rich_9,rich_0,rich_1,rich_all
count,2.077000e+03,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,...,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000
mean,3.605455e+10,0.813956,0.807262,0.838069,37733.340876,543.203659,48685.029851,751.435243,65962.064516,1148.824747,...,0.113872,0.049128,0.550668,0.385157,0.124023,0.182399,2.499278,2.498796,2.499278,2.498315
std,2.595959e+07,0.123108,0.131038,0.136805,18724.271851,148.592375,28336.768464,226.933084,39445.601512,327.845257,...,0.257604,0.154067,0.263856,0.360590,2.159801,0.339221,1.118518,1.118087,1.118518,1.098967
min,3.600500e+10,0.194083,0.183020,0.173982,4999.000000,171.000000,9893.000000,195.000000,8542.000000,231.000000,...,-0.781313,-0.731448,-0.673401,-0.789146,-0.855769,-0.732515,1.000000,1.000000,1.000000,1.000000
25%,3.604702e+10,0.755000,0.756307,0.786185,25089.000000,453.000000,30720.000000,644.000000,40031.000000,964.000000,...,-0.057997,-0.026818,0.412884,0.156717,-0.068332,-0.034494,1.000000,1.000000,1.000000,2.000000
50%,3.604712e+10,0.848471,0.846033,0.879284,36221.000000,531.000000,42483.000000,740.000000,56667.000000,1120.000000,...,0.090053,0.039649,0.526224,0.336591,0.011448,0.138452,2.000000,2.000000,2.000000,2.000000
75%,3.608103e+10,0.904543,0.896749,0.935426,46843.000000,628.000000,58833.000000,833.000000,80016.000000,1305.000000,...,0.267233,0.108568,0.655589,0.545560,0.125589,0.341604,3.000000,3.000000,3.000000,3.000000
max,3.608503e+10,0.982500,0.980721,0.998777,150001.000000,1001.000000,200001.000000,2001.000000,250001.000000,2001.000000,...,1.215991,1.477321,3.600000,3.219719,96.619048,3.156540,4.000000,4.000000,4.000000,4.000000


In [4]:
df1.shape

(2077, 30)

## df_normed0: civic_census.csv
- normalized on df1
- normalized method: (x-max)/(max-min)

In [5]:
df_normed0 = pd.read_csv('civic_census_normed.csv')
df_normed0.drop(['Unnamed: 0'],axis = 1, inplace = True)
df_normed0.describe()

,tract,entropy_index_9,entropy_index_0,entropy_index_1,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,...,rent_burden_9_0,entropy_0_1,rent_0_1,inc_0_1,rent_units_0_1,rent_burden_0_1,rich_9,rich_0,rich_1,rich_all
count,2.077000e+03,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,...,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000,2077.000000
mean,3.605455e+10,0.786224,0.782551,0.805155,0.225751,0.448438,0.204053,0.308104,0.237805,0.518545,...,0.448197,0.353398,0.286439,0.292926,0.010052,0.235254,2.499278,2.498796,2.499278,2.498315
std,2.595959e+07,0.156146,0.164269,0.165866,0.129131,0.179027,0.149056,0.125655,0.163364,0.185223,...,0.128976,0.069753,0.061744,0.089948,0.022158,0.087224,1.118518,1.118087,1.118518,1.098967
min,3.600500e+10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,3.604702e+10,0.711447,0.718674,0.742249,0.138550,0.339759,0.109554,0.248616,0.130411,0.414124,...,0.362146,0.319015,0.254197,0.235943,0.008078,0.179483,1.000000,1.000000,1.000000,2.000000
50%,3.604712e+10,0.830002,0.831154,0.855125,0.215321,0.433735,0.171429,0.301772,0.199309,0.502260,...,0.436271,0.349107,0.280719,0.280812,0.008897,0.223953,2.000000,2.000000,2.000000,2.000000
75%,3.608103e+10,0.901122,0.894732,0.923192,0.288575,0.550602,0.257433,0.353267,0.296009,0.606780,...,0.524981,0.380310,0.310991,0.332939,0.010068,0.276190,3.000000,3.000000,3.000000,3.000000
max,3.608503e+10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000,4.000000,4.000000


In [6]:
df_normed0.shape

(2077, 30)

# df2: civic_census_dropoutliers.csv
Drop outliers, based on df1

In [7]:
df2 = pd.read_csv('civic_census_dropoutliers.csv')
df2.drop(['Unnamed: 0'],axis = 1, inplace = True)
df2.shape

(1770, 30)

In [8]:
df2.describe()

,tract,entropy_index_9,entropy_index_0,entropy_index_1,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,...,rent_burden_9_0,entropy_0_1,rent_0_1,inc_0_1,rent_units_0_1,rent_burden_0_1,rich_9,rich_0,rich_1,rich_all
count,1.770000e+03,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,...,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000
mean,3.605516e+10,0.840303,0.832567,0.864719,36120.974576,537.114124,44877.863277,726.770056,60392.597740,1113.827684,...,0.121738,0.043663,0.540331,0.363513,0.060992,0.176214,2.517514,2.535593,2.544633,2.532203
std,2.625915e+07,0.092158,0.095306,0.105408,13474.715818,131.700452,18525.528534,168.262180,26936.785567,283.385431,...,0.229993,0.115547,0.215283,0.295008,0.326935,0.276348,1.061148,1.067072,1.069378,1.047937
min,3.600500e+10,0.425050,0.418577,0.183048,8377.000000,171.000000,11275.000000,195.000000,10558.000000,297.000000,...,-0.483017,-0.665910,-0.673401,-0.480250,-0.855769,-0.732515,1.000000,1.000000,1.000000,1.000000
25%,3.604702e+10,0.784663,0.784245,0.817379,26165.250000,462.000000,31361.250000,654.000000,40953.500000,967.000000,...,-0.041078,-0.019724,0.416151,0.158257,-0.061021,-0.013559,2.000000,2.000000,2.000000,2.000000
50%,3.604710e+10,0.865361,0.857691,0.894858,36022.500000,531.000000,42089.500000,738.000000,55841.500000,1111.500000,...,0.099959,0.039299,0.524086,0.333353,0.013811,0.145933,3.000000,3.000000,3.000000,3.000000
75%,3.608104e+10,0.909618,0.902633,0.940471,45403.750000,620.000000,56000.000000,816.000000,75378.000000,1265.000000,...,0.267592,0.102329,0.645497,0.522526,0.117399,0.333832,3.000000,3.000000,3.000000,3.000000
max,3.608503e+10,0.982500,0.980721,0.998777,88693.000000,973.000000,133234.000000,1418.000000,183092.000000,2001.000000,...,0.884027,0.501654,1.320388,1.445583,6.218750,1.192258,4.000000,4.000000,4.000000,4.000000


## df_normed: civic_census_normed_dropoutliers.csv
normalized on df2

In [9]:
df_normed = pd.read_csv('civic_census_normed_dropoutliers.csv')
df_normed.drop(['Unnamed: 0'],axis = 1, inplace = True)
df_normed.shape

(1770, 30)

In [10]:
df_normed.describe()

,tract,entropy_index_9,entropy_index_0,entropy_index_1,inc_9,rent_9,inc_0,rent_0,inc_1,rent_1,...,rent_burden_9_0,entropy_0_1,rent_0_1,inc_0_1,rent_units_0_1,rent_burden_0_1,rich_9,rich_0,rich_1,rich_all
count,1.770000e+03,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,...,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000
mean,3.605516e+10,0.744915,0.736447,0.835659,0.345435,0.456501,0.275526,0.434808,0.288839,0.479359,...,0.442381,0.607738,0.608756,0.438129,0.129586,0.472123,2.517514,2.535593,2.544633,2.532203
std,2.625915e+07,0.165320,0.169541,0.129219,0.167771,0.164215,0.151900,0.137582,0.156125,0.166306,...,0.168241,0.098964,0.107977,0.153185,0.046213,0.143574,1.061148,1.067072,1.069378,1.047937
min,3.600500e+10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,3.604702e+10,0.645104,0.650487,0.777626,0.221478,0.362843,0.164697,0.375307,0.176171,0.393192,...,0.323281,0.553448,0.546473,0.331548,0.112340,0.373528,2.000000,2.000000,2.000000,2.000000
50%,3.604710e+10,0.789866,0.781141,0.872607,0.344209,0.448878,0.252663,0.443990,0.262461,0.477993,...,0.426450,0.604000,0.600609,0.422468,0.122917,0.456391,3.000000,3.000000,3.000000,3.000000
75%,3.608104e+10,0.869258,0.861088,0.928523,0.461013,0.559850,0.366722,0.507768,0.375694,0.568075,...,0.549075,0.657985,0.661503,0.520698,0.137560,0.554012,3.000000,3.000000,3.000000,3.000000
max,3.608503e+10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000,4.000000,4.000000


# 2. Regression analysis on rent growth
## 2.1 analysis on every 10 years
- Use the data of 1990 and change data of 1990-2000 to be train set
- Use the data of 2000 and change data of 2000-2010 to be test set

In [11]:
columns = ['inc', 'rent', 'entropy', 'units', 
           'inc_change', 'entropy_change','units_change', 'rent_change']

In [12]:
train = pd.concat([df_normed.inc_9, df_normed.rent_9, df_normed.entropy_index_9, 
                   df_normed.r_units_9,
                  df_normed.inc_9_0, df_normed.entropy_9_0, df_normed.rent_units_9_0,
                  df_normed.rent_9_0], axis = 1)
train.columns = columns

In [13]:
test = pd.concat([df_normed.inc_0, df_normed.rent_0, df_normed.entropy_index_0, 
                   df_normed.r_units_0,
                  df_normed.inc_0_1, df_normed.entropy_0_1, df_normed.rent_units_0_1,
                  df_normed.rent_0_1], axis = 1)
test.columns = columns

In [14]:
train.shape, test.shape

((1770, 8), (1770, 8))

In [15]:
train.describe()

,inc,rent,entropy,units,inc_change,entropy_change,units_change,rent_change
count,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000
mean,0.345435,0.456501,0.744915,0.244249,0.354768,0.471086,0.395493,0.501524
std,0.167771,0.164215,0.165320,0.188100,0.143212,0.120521,0.109776,0.125795
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.221478,0.362843,0.645104,0.098950,0.257861,0.406323,0.334385,0.429386
50%,0.344209,0.448878,0.789866,0.196932,0.337636,0.466949,0.372814,0.501468
75%,0.461013,0.559850,0.869258,0.336926,0.430760,0.532481,0.432716,0.570857
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
test.describe()

,inc,rent,entropy,units,inc_change,entropy_change,units_change,rent_change
count,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000
mean,0.275526,0.434808,0.736447,0.259952,0.438129,0.607738,0.129586,0.608756
std,0.151900,0.137582,0.169541,0.188245,0.153185,0.098964,0.046213,0.107977
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.164697,0.375307,0.650487,0.114268,0.331548,0.553448,0.112340,0.546473
50%,0.252663,0.443990,0.781141,0.215488,0.422468,0.604000,0.122917,0.600609
75%,0.366722,0.507768,0.861088,0.355920,0.520698,0.657985,0.137560,0.661503
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
model = smf.ols('rent_change ~ '+ '+'.join(train.columns[:-1]) ,train).fit()
model_1 = smf.ols('rent_change ~ '+ '+'.join(test.columns[:-1]) ,test).fit()

In [18]:
total = pd.concat([train,test], axis = 0)
total.shape

(3540, 8)

In [19]:
model_2 = smf.ols('rent_change ~ '+ '+'.join(total.columns[:-1]) ,total).fit()

# Multi-regression RESULT
## Do regression during every 10 years

## 1990-2000 regression result

In [20]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rent_change   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     103.4
Date:                Fri, 01 Dec 2017   Prob (F-statistic):          7.47e-127
Time:                        10:56:41   Log-Likelihood:                 1462.8
No. Observations:                1770   AIC:                            -2910.
Df Residuals:                    1762   BIC:                            -2866.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept          0.2291      0.031      7.401      0.000         0.168     0.290
inc                0.4527      0.024     18.512      0.000         0.405     0.501
rent              -0.4755      0.024    -19.669      0.000        -0.523    -0.428
entropy            0.2097      0.019     11.156      0.000         0.173     0.247
units              0.1170      0.016      7.270      0.000         0.085     0.149
inc_change         0.2817      0.019     14.678      0.000         0.244     0.319
entropy_change     0.1095      0.025      4.402      0.000         0.061     0.158
units_change      -0.0080      0.025     -0.313      0.754        -0.058     0.042
==============================================================================
Omnibus:                       84.883   Durbin-Watson:                   1.713
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              288.312
Skew:                           0.052   Prob(JB):                     2.48e-63
Kurtosis:                       4.974   Cond. No.                         27.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2000-2010 regression result

In [21]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rent_change   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     16.28
Date:                Fri, 01 Dec 2017   Prob (F-statistic):           6.92e-21
Time:                        10:56:41   Log-Likelihood:                 1484.2
No. Observations:                1770   AIC:                            -2952.
Df Residuals:                    1762   BIC:                            -2909.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept          0.5718      0.030     19.242      0.000         0.514     0.630
inc                0.0826      0.032      2.618      0.009         0.021     0.144
rent              -0.1784      0.029     -6.226      0.000        -0.235    -0.122
entropy            0.0215      0.019      1.125      0.261        -0.016     0.059
units             -0.0046      0.015     -0.309      0.757        -0.034     0.024
inc_change         0.1259      0.017      7.519      0.000         0.093     0.159
entropy_change     0.0017      0.030      0.056      0.955        -0.057     0.061
units_change       0.1612      0.055      2.912      0.004         0.053     0.270
==============================================================================
Omnibus:                       80.193   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              261.660
Skew:                           0.044   Prob(JB):                     1.52e-57
Kurtosis:                       4.882   Cond. No.                         35.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 1990-2000 + 2000-2010  regression result

In [22]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rent_change   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     189.8
Date:                Fri, 01 Dec 2017   Prob (F-statistic):          1.89e-239
Time:                        10:56:41   Log-Likelihood:                 2795.3
No. Observations:                3540   AIC:                            -5575.
Df Residuals:                    3532   BIC:                            -5525.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept          0.4415      0.022     20.097      0.000         0.398     0.485
inc                0.2669      0.019     14.094      0.000         0.230     0.304
rent              -0.3365      0.019    -17.941      0.000        -0.373    -0.300
entropy            0.1284      0.014      9.400      0.000         0.102     0.155
units              0.0316      0.011      2.846      0.004         0.010     0.053
inc_change         0.2108      0.013     16.189      0.000         0.185     0.236
entropy_change     0.1094      0.017      6.254      0.000         0.075     0.144
units_change      -0.2469      0.014    -17.664      0.000        -0.274    -0.219
==============================================================================
Omnibus:                      150.022   Durbin-Watson:                   1.769
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              472.255
Skew:                           0.077   Prob(JB):                    2.83e-103
Kurtosis:                       4.783   Cond. No.                         26.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### use the first 10 years to test latter 10 years

In [23]:
train.head(2)

,inc,rent,entropy,units,inc_change,entropy_change,units_change,rent_change
0,0.377683,0.711970,0.793073,0.105030,0.388886,0.464519,0.384006,0.226547
1,0.399758,0.453865,0.887558,0.058043,0.211059,0.482413,0.949193,0.383516


In [24]:
test.head(2)

,inc,rent,entropy,units,inc_change,entropy_change,units_change,rent_change
0,0.324224,0.437449,0.781640,0.115600,0.326530,0.633739,0.192816,0.710826
1,0.243885,0.366312,0.897852,0.133838,0.656091,0.588810,0.158484,0.873628


In [25]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [26]:
def modelEval(lm, test = test, key = 'Y'):
    lmy = lm.predict(test)
    y_err = lmy - test[key]
    y_norm = test[key]-np.mean(test[key])
    return 1-y_err.dot(y_err)/y_norm.dot(y_norm)


In [27]:
R_2_IS_OLS = model.rsquared
R_2_OS_OLS = modelEval(model, key = 'rent_change')
OLS_coef = model.params

R_2_IS_OLS, R_2_OS_OLS

(0.29109681422116163, -0.75516345107303517)

### out-of-sample R2 is less than 0. this model sucks.

### Try Ridge 

In [28]:
def modelEval2(lm, X_test = X_test, y_test = y_test):
    y_err = lm.predict(X_test) - y_test
    y_norm = y_test - np.mean(y_test)
    Rsquared = 1- y_err.dot(y_err)/y_norm.dot(y_norm)
    return Rsquared

In [29]:
#Find the Alpha and report best test performance for Ridge/Lasso.
def Regularization_fit_lambda(model,X_train,y_train,lambdas):
    R_2_OS=[]
    if model==1:
        RM = lambda a: linear_model.Ridge(fit_intercept=True, alpha=a)
        model_label='Ridge'
    else:
        RM = lambda a: linear_model.Lasso(fit_intercept=True, alpha=a)
        model_label='Lasso'
    
    best_R2 = -1
    best_lambda = lambdas[0]
    
    for i in lambdas:
        lm = RM(i)
        lm.fit(X_train,y_train)  
        R_2_OS_ = modelEval2(lm, X_test, y_test)
        R_2_OS.append(R_2_OS_)
        
        if R_2_OS_ > best_R2:
            best_R2 = R_2_OS_
            best_lambda = i
    
    return best_lambda

In [30]:
#select best lambda for Ridge
lambdas = np.exp(np.linspace(-5,10,200))
lambda_r_optimal=Regularization_fit_lambda(1,X_train,y_train,lambdas)
print('Optimal lambda for Ridge={0}'.format(lambda_r_optimal))

Optimal lambda for Ridge=10.880091739883493


In [31]:
Ridge = linear_model.Ridge(fit_intercept=True,alpha=lambda_r_optimal)

Ridge.fit(X_train,y_train)

Ridge_coef=Ridge.coef_
R_2_IS_Ridge = modelEval2(Ridge, X_train, y_train)
R_2_OS_Ridge = modelEval2(Ridge, X_test, y_test)

print("The R-squared we found for IS Ridge is: {0}".format(R_2_IS_Ridge))
print("The R-squared we found for OS Ridge is: {0}".format(R_2_OS_Ridge))

The R-squared we found for IS Ridge is: 0.2375039057238466
The R-squared we found for OS Ridge is: -0.6459548019596499


In [32]:
#let's look at the final coefficients
name=list(OLS_coef.index)[1:]

result=[name, np.ndarray.tolist(np.asarray(OLS_coef[1:])),
        np.ndarray.tolist(Ridge_coef)] 

temp=pd.DataFrame(result).T

R2_IS=pd.DataFrame(['R2_train',R_2_IS_OLS, R_2_IS_Ridge]).T 
R2_OS=pd.DataFrame(['R2_test',R_2_OS_OLS, R_2_OS_Ridge]).T 


temp=temp.append([R2_IS, R2_OS])
result=temp

result.columns=['','OLS','Ridge'] 
result.set_index('')

,OLS,Ridge
,,
inc,0.452736,0.199635
rent,-0.475543,-0.24822
entropy,0.209668,0.0885395
units,0.117003,0.102729
inc_change,0.281725,0.164327
entropy_change,0.109479,0.0122442
units_change,-0.00797249,-0.0458599
R2_train,0.291097,0.237504
R2_test,-0.755163,-0.645955


## Ridge regression does not get a better result than OLS

## 2.2 analysis on 20 years together
Use 20 years data together to do train and test

In [33]:
from sklearn.model_selection import train_test_split
train2, test2  = train_test_split(total, test_size = 0.3)

X_train2 = train2.iloc[:,:-1]
y_train2 = train2.iloc[:,-1]
X_test2 = test2.iloc[:,:-1]
y_test2 = test2.iloc[:,-1]
X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape

((2478, 7), (2478,), (1062, 7), (1062,))

In [34]:
model2 = smf.ols('rent_change ~ '+ '+'.join(X_train2.columns) ,train2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rent_change   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     133.1
Date:                Fri, 01 Dec 2017   Prob (F-statistic):          1.41e-166
Time:                        10:56:41   Log-Likelihood:                 1915.5
No. Observations:                2478   AIC:                            -3815.
Df Residuals:                    2470   BIC:                            -3768.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept          0.4390      0.027     16.271      0.000         0.386     0.492
inc                0.2668      0.023     11.591      0.000         0.222     0.312
rent              -0.3483      0.023    -15.261      0.000        -0.393    -0.304
entropy            0.1310      0.017      7.859      0.000         0.098     0.164
units              0.0290      0.014      2.113      0.035         0.002     0.056
inc_change         0.2086      0.016     13.222      0.000         0.178     0.239
entropy_change     0.1220      0.021      5.769      0.000         0.081     0.163
units_change      -0.2414      0.017    -14.108      0.000        -0.275    -0.208
==============================================================================
Omnibus:                      106.130   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              339.650
Skew:                           0.040   Prob(JB):                     1.76e-74
Kurtosis:                       4.812   Cond. No.                         26.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
OLS_coef2 = model2.params
R_2_IS_OLS2 = model2.rsquared
R_2_OS_OLS2 = modelEval(model2, test = test2, key = 'rent_change')
R_2_IS_OLS2, R_2_OS_OLS2

(0.27388111633119594, 0.27009176063382934)

In [36]:
lambdas = np.exp(np.linspace(-5,10,200))
lambda_r_optimal2 = Regularization_fit_lambda(1,X_train2,y_train2,lambdas)
print('Optimal lambda for Ridge={0}'.format(lambda_r_optimal2))


Optimal lambda for Ridge=12.650395028192444


In [37]:
Ridge2 = linear_model.Ridge(fit_intercept=True,alpha=lambda_r_optimal2)

Ridge2.fit(X_train2,y_train2)

Ridge_coef2 = Ridge2.coef_
R_2_IS_Ridge2 = modelEval2(Ridge2, X_train2, y_train2)
R_2_OS_Ridge2 = modelEval2(Ridge2, X_test2, y_test2)

print("The R-squared we found for IS Ridge is: {0}".format(R_2_IS_Ridge2))
print("The R-squared we found for OS Ridge is: {0}".format(R_2_OS_Ridge2))

The R-squared we found for IS Ridge is: 0.25335596431201013
The R-squared we found for OS Ridge is: 0.2513370092994037


In [38]:
name=list(OLS_coef2.index)[1:] 

result = [name, np.ndarray.tolist(np.asarray(OLS_coef2[1:])),
        np.ndarray.tolist(Ridge_coef2)]
temp = pd.DataFrame(result).T
R2_IS2 = pd.DataFrame(['R2_train', R_2_IS_OLS2, R_2_IS_Ridge2]).T 
R2_OS2 = pd.DataFrame(['R2_test', R_2_OS_OLS2, R_2_OS_Ridge2]).T 

temp = temp.append([R2_IS2, R2_OS2])
result = temp
result.columns=['','OLS2','Ridge2']
result.set_index('')

,OLS2,Ridge2
,,
inc,0.266771,0.104228
rent,-0.348309,-0.192224
entropy,0.130981,0.0570592
units,0.0290166,0.030921
inc_change,0.208565,0.153662
entropy_change,0.12201,0.0820625
units_change,-0.241378,-0.211795
R2_train,0.273881,0.253356
R2_test,0.270092,0.251337


# Multi-linear regression on rent growth analysis conclusion

### Fomular: Rent change ~ Rent + Income + Income change + entropy index + entropy index change + rent units + rent units change

#### Data prepared: normalized 

### Regression results explanation:
- Rsquared of 1990-2000, and 1990-2000 + 2000-2010 total , is between **0.27-0.3**. However, 2000-2010's Rsquared is pretty low, roughly 0.06. It means that the rent growth are less related to the income, entropy index and number of renter units than before. (**Is that because the rent market has faced more governmental interference recently, like rent control,  rent stabilization, inclusionary zoning programs, public housing, etc.? **)



- **Income, rent, and income change are always the primary factors** influencing the rent growth.


- Rent has the main negative effect on rent growth. Higher rent, slower rent growth, which is obvious.


- The number change of rent units also has negative influence on rent growth between 1990-2000, but not statistically significant(p>0.7).

- Entropy index, entropy index change, as well as the number of rent units are not statistically significant in 2000-2010(p>0.2, 0.9, 0.7).

- Generally, **it's hard to find relationship between rent growth with either entropy index, or entropy index change(the coef efficiencies are small)**. They have weak positive corelation with rent growth between 1990-2000, but nearly no corelation relationship between 2000-2010.

### Train-test results:
- The result is awful if use 1990-2000 model to predict 2000-2010 rent growth. Test Rsquared is negative.

- For the dataset of 1990-2000 + 2000-2010 total, if splited into train-test set, the Rsquared for train and test are both about 0.27. 



# 3. Regression analysis on rent

# Regression: Rent ~ entropy + inc + units

## 3.1 Normalized with outliers

In [39]:
df_normed0.shape

(2077, 30)

In [40]:
data0_9 = df_normed0[['tract','entropy_index_9','inc_9', 'rent_9','r_units_9','ratio_rent_inc_9' ]]
data0_0 = df_normed0[['tract','entropy_index_0','inc_0', 'rent_0','r_units_0','ratio_rent_inc_0' ]]
data0_1 = df_normed0[['tract','entropy_index_1','inc_1', 'rent_1','r_units_1','ratio_rent_inc_1' ]]
data0_9.shape, data0_0.shape, data0_1.shape

((2077, 6), (2077, 6), (2077, 6))

In [41]:
col = ['tract','entropy', 'inc', 'rent', 'units', 'burden']
data0_9.columns = col
data0_0.columns = col
data0_1.columns = col

data0_9_0 = pd.concat([data0_9, data0_0],axis = 0)
data0 = pd.concat([data0_9, data0_0, data0_1],axis = 0)
data0.shape

(6231, 6)

In [42]:
data0.head()

,tract,entropy,inc,rent,units,burden
0,36005000200,0.853692,0.232493,0.687952,0.042255,0.198349
1,36005000400,0.920498,0.244721,0.438554,0.023351,0.111026
2,36005001600,0.905587,0.136088,0.355422,0.162126,0.193571
3,36005001900,0.581888,0.136715,0.302410,0.039253,0.166555
4,36005002000,0.787263,0.094916,0.153012,0.285333,0.150166


In [43]:
lm0 = smf.ols('rent ~ entropy + inc + units', data0).fit()
lm0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     2008.
Date:                Fri, 01 Dec 2017   Prob (F-statistic):               0.00
Time:                        10:56:42   Log-Likelihood:                 3709.3
No. Observations:                6231   AIC:                            -7411.
Df Residuals:                    6227   BIC:                            -7384.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0255      0.012      2.111      0.035         0.002     0.049
entropy        0.2435      0.012     19.825      0.000         0.219     0.268
inc            0.9843      0.013     73.967      0.000         0.958     1.010
units         -0.1078      0.017     -6.302      0.000        -0.141    -0.074
==============================================================================
Omnibus:                      359.265   Durbin-Watson:                   1.228
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              504.032
Skew:                           0.519   Prob(JB):                    3.55e-110
Kurtosis:                       3.930   Cond. No.                         15.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
train3 = data0_9_0
test3 = data0_1
model3 = smf.ols('rent ~ entropy + inc + units', train3).fit()

OLS_coef3 = model3.params
R_2_IS_OLS3 = model3.rsquared
R_2_OS_OLS3 = modelEval(model3, test = test3, key = 'rent')
R_2_IS_OLS3, R_2_OS_OLS3

(0.52847432600236743, 0.10114649988673319)

## 3.2 Normalized and no outliers

In [45]:
data_9 = df_normed[['tract','entropy_index_9','inc_9', 'rent_9','r_units_9','ratio_rent_inc_9' ]]
data_0 = df_normed[['tract','entropy_index_0','inc_0', 'rent_0','r_units_0','ratio_rent_inc_0' ]]
data_1 = df_normed[['tract','entropy_index_1','inc_1', 'rent_1','r_units_1','ratio_rent_inc_1' ]]
data_9.shape, data_0.shape, data_1.shape

((1770, 6), (1770, 6), (1770, 6))

In [46]:
col = ['tract','entropy', 'inc', 'rent', 'units', 'burden']
data_9.columns = col
data_0.columns = col
data_1.columns = col

data_9_0 = pd.concat([data_9, data_0],axis = 0)
data = pd.concat([data_9, data_0, data_1],axis = 0)
data.shape

(5310, 6)

In [47]:
lm = smf.ols('rent ~ entropy + inc + units', data).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     1810.
Date:                Fri, 01 Dec 2017   Prob (F-statistic):               0.00
Time:                        10:56:42   Log-Likelihood:                 4148.5
No. Observations:                5310   AIC:                            -8289.
Df Residuals:                    5306   BIC:                            -8263.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0973      0.010      9.778      0.000         0.078     0.117
entropy        0.2322      0.010     23.659      0.000         0.213     0.251
inc            0.6688      0.011     63.151      0.000         0.648     0.690
units         -0.0881      0.009    -10.147      0.000        -0.105    -0.071
==============================================================================
Omnibus:                      167.488   Durbin-Watson:                   1.529
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              442.346
Skew:                           0.063   Prob(JB):                     8.83e-97
Kurtosis:                       4.408   Cond. No.                         13.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
train4 = data_9_0
test4 = data_1
model4 = smf.ols('rent ~ entropy + inc + units', train4).fit()

OLS_coef4 = model4.params
R_2_IS_OLS4 = model4.rsquared
R_2_OS_OLS4 = modelEval(model4, test = test4, key = 'rent')
R_2_IS_OLS4, R_2_OS_OLS4

(0.55822002367154122, 0.38482806084740906)

## 3.3 Not normalized and with outliers

In [49]:
data1_9 = df1[['tract','entropy_index_9','inc_9', 'rent_9','r_units_9','ratio_rent_inc_9' ]]
data1_0 = df1[['tract','entropy_index_0','inc_0', 'rent_0','r_units_0','ratio_rent_inc_0' ]]
data1_1 = df1[['tract','entropy_index_1','inc_1', 'rent_1','r_units_1','ratio_rent_inc_1' ]]
data1_9.shape, data1_0.shape, data1_1.shape

((2077, 6), (2077, 6), (2077, 6))

In [50]:
col = ['tract','entropy', 'inc', 'rent', 'units', 'burden']
data1_9.columns = col
data1_0.columns = col
data1_1.columns = col

data1_9_0 = pd.concat([data1_9, data1_0],axis = 0)
data1 = pd.concat([data1_9, data1_0, data1_1],axis = 0)
data1.shape

(6231, 6)

In [51]:
lm1 = smf.ols('rent ~ entropy + inc + units', data1).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                     2913.
Date:                Fri, 01 Dec 2017   Prob (F-statistic):               0.00
Time:                        10:56:43   Log-Likelihood:                -42634.
No. Observations:                6231   AIC:                         8.528e+04
Df Residuals:                    6227   BIC:                         8.530e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -216.9728     23.862     -9.093      0.000      -263.750  -170.196
entropy      695.5562     24.731     28.125      0.000       647.074   744.038
inc            0.0092   9.97e-05     92.200      0.000         0.009     0.009
units         -0.0056      0.003     -1.690      0.091        -0.012     0.001
==============================================================================
Omnibus:                      802.941   Durbin-Watson:                   1.134
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1400.288
Skew:                           0.858   Prob(JB):                    8.54e-305
Kurtosis:                       4.564   Cond. No.                     7.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [52]:
train5 = data1_9_0
test5 = data1_1
model5 = smf.ols('rent ~ entropy + inc + units', train5).fit()

OLS_coef5 = model5.params
R_2_IS_OLS5 = model5.rsquared
R_2_OS_OLS5 = modelEval(model5, test = test5, key = 'rent')
R_2_IS_OLS5, R_2_OS_OLS5

(0.60931339756300118, -0.48087741902360581)

## 3.4 Not normalized and no outliers

In [53]:
data2_9 = df2[['tract','entropy_index_9','inc_9', 'rent_9','r_units_9','ratio_rent_inc_9' ]]
data2_0 = df2[['tract','entropy_index_0','inc_0', 'rent_0','r_units_0','ratio_rent_inc_0' ]]
data2_1 = df2[['tract','entropy_index_1','inc_1', 'rent_1','r_units_1','ratio_rent_inc_1' ]]
data2_9.shape, data2_0.shape, data2_1.shape

((1770, 6), (1770, 6), (1770, 6))

In [54]:
col = ['tract','entropy', 'inc', 'rent', 'units', 'burden']
data2_9.columns = col
data2_0.columns = col
data2_1.columns = col

data2_9_0 = pd.concat([data2_9, data2_0],axis = 0)
data2 = pd.concat([data2_9, data2_0, data2_1],axis = 0)
data2.shape

(5310, 6)

In [55]:
lm2 = smf.ols('rent ~ entropy + inc + units', data2).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     2387.
Date:                Fri, 01 Dec 2017   Prob (F-statistic):               0.00
Time:                        10:56:43   Log-Likelihood:                -35822.
No. Observations:                5310   AIC:                         7.165e+04
Df Residuals:                    5306   BIC:                         7.168e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -431.4087     27.652    -15.602      0.000      -485.617  -377.200
entropy      822.1192     29.174     28.180      0.000       764.927   879.311
inc            0.0107      0.000     81.012      0.000         0.010     0.011
units          0.0246      0.004      5.593      0.000         0.016     0.033
==============================================================================
Omnibus:                      470.043   Durbin-Watson:                   1.069
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              712.224
Skew:                           0.681   Prob(JB):                    2.20e-155
Kurtosis:                       4.167   Cond. No.                     7.35e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.35e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
train6 = data2_9_0
test6 = data2_1
model6 = smf.ols('rent ~ entropy + inc + units', train6).fit()

OLS_coef6 = model6.params
R_2_IS_OLS6 = model6.rsquared
R_2_OS_OLS6 = modelEval(model6, test = test6, key = 'rent')
R_2_IS_OLS6, R_2_OS_OLS6

(0.5651690608529194, -0.76162251753331911)

## 3.5 Results

In [57]:
name=list(lm.params.index)

result = [name, np.ndarray.tolist(np.asarray(lm0.params)),
        np.ndarray.tolist(np.asarray(lm.params)),
         np.ndarray.tolist(np.asarray(lm1.params)),
         np.ndarray.tolist(np.asarray(lm2.params))]
temp = pd.DataFrame(result).T
R2 = pd.DataFrame(['R2', lm0.rsquared, lm.rsquared, lm1.rsquared, lm2.rsquared]).T 

R2_IS2 = pd.DataFrame(['R2_train', R_2_IS_OLS3, R_2_IS_OLS4, R_2_IS_OLS5, R_2_IS_OLS6]).T 
R2_OS2 = pd.DataFrame(['R2_test', R_2_OS_OLS3, R_2_OS_OLS4, R_2_OS_OLS5, R_2_OS_OLS6]).T 

temp = temp.append([R2, R2_IS2, R2_OS2])
result = temp
result.columns=['','Normalized','Normalized without outliers', 'Original', 'Original without outliers']
result.set_index('')

,Normalized,Normalized without outliers,Original,Original without outliers
,,,,
Intercept,0.0255446,0.0973064,-216.973,-431.409
entropy,0.243453,0.232189,695.556,822.119
inc,0.984299,0.668796,0.00919218,0.0107321
units,-0.107781,-0.0881293,-0.00563547,0.0245921
R2,0.491744,0.505835,0.583902,0.57438
R2_train,0.528474,0.55822,0.609313,0.565169
R2_test,0.101146,0.384828,-0.480877,-0.761623


### Multi-linear regression analysis conclusion: Normalized without outliers is pretty good. The test Rsquared is 0.385
#### Fomular: Rent ~ Income + entropy index  + rent units
#### Train: 1990, 2000
#### Test: 2010
